## Connect to your Worksace

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [2]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Prepare Data

In [3]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input
import mltable

# creates a dataset based on the files in the local data folder
training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:obesity:1")

## Configure ML Job

In [4]:
from azure.ai.ml.automl import ClassificationPrimaryMetrics
 
list(ClassificationPrimaryMetrics)

[<ClassificationPrimaryMetrics.AUC_WEIGHTED: 'AUCWeighted'>,
 <ClassificationPrimaryMetrics.ACCURACY: 'Accuracy'>,
 <ClassificationPrimaryMetrics.NORM_MACRO_RECALL: 'NormMacroRecall'>,
 <ClassificationPrimaryMetrics.AVERAGE_PRECISION_SCORE_WEIGHTED: 'AveragePrecisionScoreWeighted'>,
 <ClassificationPrimaryMetrics.PRECISION_SCORE_WEIGHTED: 'PrecisionScoreWeighted'>]

In [5]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="ci123123",                 # Nombre del cluster de cómputo
    experiment_name="exp-obesitydata-basic", # Nombre que le quieres dar al experimento
    training_data=training_data_input,             # Data / Dataset de input
    target_column_name="NObeyesdad",                 # Columna objetivo de la clasificación
    primary_metric="accuracy",                     # Metrica de evaluación
    n_cross_validations=5,                         # Número de validaciones cruzadas
    enable_model_explainability=True               # Permite las explicaciones de las predicciones
)

In [6]:
# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

In [7]:
# set the training properties (optional)
classification_job.set_training(
    #enable_onnx_compatible_models=True
)

## Run an automated machine learning job

In [8]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/dreamy_feast_ybtzb1r6h7?wsid=/subscriptions/08f17054-76c9-49db-b287-b10383c293e3/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs&tid=1b604f7d-1cee-45b6-8431-4090d7becc32
